In [1]:
%matplotlib widget
import numpy as np
import networkx as nx
import random
import matplotlib.pyplot as plt
import sys
import warnings
from tqdm import tqdm  # For progress bar

# --- IMPORTS FOR ML ---
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# --- USER PATH ---
from RGG_Library import RGGBuilder

# --------------------------------------------------------------------------------
# CONFIGURATION
# --------------------------------------------------------------------------------
space_ = "torus"
use_angles = True
perturb_radius_multiplier_ = 1.2

# Experiment Range: 0.0, then 0.02 to 1.5
perturb_values = np.concatenate(([0.0], np.arange(1000, 11000, 1000)))

# Fixed Graph Parameters
N_NODES = 1024
K_NEIGHBORS = 20
N_SAMPLES = 2000
LASSO_ALPHA = 1e-6

# --------------------------------------------------------------------------------
# SIMULATION FUNCTION
# --------------------------------------------------------------------------------
def get_coefficients_for_perturbation(perturb_val, iteration_seed):
    """
    Generates a graph with specific perturbation, runs Lasso, 
    and returns the dictionary of Real (Unscaled) Coefficients.
    """
    
    # --- LOGIC UPDATE: Handle p=0 explicitly ---
    if perturb_val == 0.0:
        is_perturbed = False
        current_scale = 0.0 # Placeholder, won't be used
    else:
        is_perturbed = True
        current_scale = perturb_val
    
    builder = RGGBuilder(
        n=N_NODES, 
        k=K_NEIGHBORS, 
        connectivity_regime="sc", 
        space=space_, 
        order=4, 
        perturb=is_perturbed, 
        perturb_scale=current_scale, 
        perturb_radius_multiplier=perturb_radius_multiplier_, 
        seed=iteration_seed
    )
    
    G = builder.build()

    # Ensure Giant Component
    if not nx.is_connected(G):
        components = list(nx.connected_components(G))
        Gsub = G.subgraph(max(components, key=len)).copy()
    else:
        Gsub = G
    
    Gsub = nx.convert_node_labels_to_integers(Gsub, ordering="sorted")

    # 2. Sample Data
    random.seed(iteration_seed)
    np.random.seed(iteration_seed)
    
    if use_angles:
        res, degs, dists, pairs, angles = RGGBuilder.sample_commute_times_even_distance_w_angles(
            Gsub, nsamples=N_SAMPLES, n_bins=20, seed=iteration_seed, 
            min_dist=builder.radius, max_dist=2
        )
    else:
        res, degs, dists, pairs = RGGBuilder.sample_commute_times_even_distance(
            Gsub, nsamples=N_SAMPLES, n_bins=20, seed=iteration_seed, 
            min_dist=builder.radius, max_dist=2
        )
        angles = np.zeros_like(dists)

    # 3. Feature Engineering
    safe_dists = dists.copy()
    safe_dists[safe_dists == 0] = 1e-9 


    if is_perturbed == False:
        # --- PURE GEOMETRIC FEATURES ONLY ---
        feature_dict = {
            "Log(d)": np.log(safe_dists),
            "d^2": dists**2,
            
            # Global Geometry
            "d^4 * cos(4theta)": dists**4 * np.cos(4*angles),
            "d^8 * cos(8theta)": dists**8 * np.cos(8*angles),
            #"d^12 * cos(12theta)": dists**12 * np.cos(12*angles),
            
            # Local Artifacts (Should stay zero/low)
            "1/d^4 * cos(4theta)": (1/(safe_dists**4)) * np.cos(4*angles),
            #"1/d^8 * cos(8theta)": (1/(safe_dists**8)) * np.cos(8*angles),
            #"1/d^12 * cos(12theta)": (1/(safe_dists**12)) * np.cos(12*angles),
        }
    
    else:
        feature_dict = {
            "Log(d)": np.log(safe_dists),
            "d^2": dists**2,
            
            # Global Geometry
            "d^4 * cos(4theta)": dists**4 * np.cos(4*angles),
            "d^8 * cos(8theta)": dists**8 * np.cos(8*angles),
            
            # Local Artifacts (Should stay zero/low)
            "1/d^4 * cos(4theta)": (1/(safe_dists**4)) * np.cos(4*angles),

            # New Regime
            "InvDegSum": degs,
        }

    feature_names = list(feature_dict.keys())
    X = np.column_stack(list(feature_dict.values()))
    y = res

    # 4. Lasso Regression
    model = make_pipeline(
        StandardScaler(), 
        Lasso(alpha=LASSO_ALPHA, max_iter=50000, tol=1e-4)
    )
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        model.fit(X, y)

    # 5. Extract Unscaled Coefficients
    lasso = model.named_steps['lasso']
    scaler = model.named_steps['standardscaler']
    
    sigmas = scaler.scale_
    sigmas[sigmas == 0] = 1.0 
    
    real_coefs = lasso.coef_ / sigmas
    
    return dict(zip(feature_names, real_coefs))

# --------------------------------------------------------------------------------
# MAIN EXECUTION LOOP
# --------------------------------------------------------------------------------

history = {key: [] for key in [
    "Log(d)", 
    "d^2", 
    "d^4 * cos(4theta)", 
    "d^8 * cos(8theta)", 
    "1/d^4 * cos(4theta)",
    "InvDegSum"
]}

print(f"Starting Geometric Experiment: {len(perturb_values)} steps.")
print("Step 1: Perturb=False (Perfect Lattice)")
print("Steps 2+: Perturb=True (Scale epsilon -> 1.5)")

perturb_vals_plotted = []

for i, p in enumerate(tqdm(perturb_values)):
    try:
        coefs = get_coefficients_for_perturbation(p, iteration_seed=0)
        
        for key in history.keys():
            history[key].append(coefs.get(key, 0.0))
        
        perturb_vals_plotted.append(p)
            
    except Exception as e:
        print(f"Skipping p={p:.2f} due to error: {e}")

# --------------------------------------------------------------------------------
# VISUALIZATION
# --------------------------------------------------------------------------------

features_to_plot = list(history.keys())
n_feats = len(features_to_plot)
cols = 3
rows = (n_feats // cols) + (1 if n_feats % cols > 0 else 0)

fig, axes = plt.subplots(rows, cols, figsize=(14, 3.5 * rows), constrained_layout=True)
axes = axes.flatten()

for i, feature_name in enumerate(features_to_plot):
    ax = axes[i]
    y_vals = history[feature_name]
    
    # Color coding
    color = 'tab:blue'
    if "Log" in feature_name: color = 'tab:green'      # Source
    elif "1/" in feature_name: color = 'tab:red'       # Artifact
    elif "^2" in feature_name: color = 'tab:orange'    # Background
    
    ax.plot(perturb_vals_plotted, y_vals, marker='o', markersize=3, linewidth=1.5, color=color, alpha=0.8)
    ax.axhline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)
    
    ax.set_title(feature_name, fontsize=11, fontweight='bold')
    ax.set_xlabel("Perturbation Scale")
    ax.set_ylabel("Coefficient Value")
    ax.grid(True, alpha=0.3)

for j in range(i + 1, len(axes)):
    axes[j].axis('off')

plt.suptitle(f"Evolution of Geometric Coefficients (p=0 is Perfect Lattice)\n(Lasso Alpha={LASSO_ALPHA})", fontsize=16)
plt.show()

Starting Geometric Experiment: 11 steps.
Step 1: Perturb=False (Perfect Lattice)
Steps 2+: Perturb=True (Scale epsilon -> 1.5)


  0%|          | 0/11 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [2]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
